In [2]:
# !pip install mido
# !pip install keras

In [3]:
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import numpy as np

/opt/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [7]:
mid = MidiFile('data/Nintendo_-_Pokemon_Fire_Red_Route_1_Piano_Cover_Hard_Version.mid')
notes = []
velocities = []

In [8]:
for msg in mid:
    if not msg.is_meta:
        if msg.channel == 0:
            if msg.type == 'note_on':
                data = msg.bytes()
                note = data[1]
                velocity = data[2]
                notes.append(note)
                velocities.append(velocity)
                
combine = [[i,j] for i,j in zip(notes, velocities)]
                

In [ ]:
# print()

In [10]:
note_min = np.min(notes)
note_max = np.max(notes)
velocities_min = np.min(velocities)
velocities_max = np.max(velocities)

for i in combine:
    i[0] = 2*(i[0]-((note_min+note_max)/2))/(note_max-note_min)
    i[1] = 2*(i[1]-((velocities_min+velocities_max)/2))/(velocities_max-velocities_min)


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  


In [11]:
X = []
Y = []
n_prev = 30
for i in range(len(combine)-n_prev):
    x = combine[i:i+n_prev]
    y = combine[i+n_prev]
    X.append(x)
    Y.append(y)
    
seed = combine[0:n_prev]

In [13]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev,2), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128, input_shape=(n_prev,2), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, input_shape=(n_prev,2), return_sequences=True))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('linear'))
optimizer = Adam(lr=1e-3)
model.compile(loss='mse', optimizer=optimizer)
filepath = "Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)


In [14]:
# model.fit(np.array(X), np.array(Y), 32, 5, verbose=1, callbacks=[model_save_callback])

In [ ]:
#Prediction!

prediction = []
x = seed
x = np.expand_dims(x,axis=0)

for i in range(300):
    preds = model.predict(x)
    x = np.sqeeze(x)
    x = np.concatenate((x, preds))
    x = x[1:]
    x = np.expand_dims(x, axix = 0)
    preds = np.sqeeze(preds)
    prediction.append(preds)

In [1]:
for pred in prediction:
    pred[0] = int((pred[0]/2)*(note_max-note_min) + (note_min+note_max)/2)
    pred[1] = int((pred[1]/2)*(velocities_max-velocities_min) + (velocities_min+velocities_man)/2)
    if pred[0] < 24:
        pred[0] = 24
    elif pred[0] > 102:
        pred[0] = 102
    if pred[1] < 0:
        pred[1] = 0
    elif pred[1] > 127:
        pred[1] = 127

NameError: name 'prediction' is not defined

In [ ]:
mid = MidiFile()
track = MidiTrack()

t=0
for note in prediction:
    note = np.asarray([147, note[0], note[1]])
    byte = note.astype(int)
    msg = Message.from_bytes(byte[0:3])
    t+=1
    msg.time = t
    track.append(msg)
    
mid.tracks.append(track)
mid.save('Generated_song_epoch =10.mid')